In [ ]:
# iii = ['iii']*999
# while True:
#   iii.append(iii)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip 'drive/MyDrive/data.zip'

In [ ]:
!pip install pydicom

In [ ]:
!pip install xlsxwriter

In [ ]:
import os
import numpy as np
import random
import tensorflow as tf

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [ ]:
import cv2
import pydicom
import xlsxwriter
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LambdaCallback
from math import sqrt
from os import walk
from random import random, randint
from copy import deepcopy
from scipy.ndimage import rotate

In [ ]:
CASE_SE = 'Without-scSE'
RES_FOLDER = f'drive/MyDrive/SRN-results/{CASE_SE}'

# Створюємо папки, якщо їх немає
os.makedirs(RES_FOLDER, exist_ok=True)
os.makedirs(f'{RES_FOLDER}/imgs', exist_ok=True)

NUM_EPOCHS = 100
BATCH_SIZE = 16
LEARNING_RATE = 0.00005

HP = {
    'CASE_SE': CASE_SE,
    'NUM_EPOCHS': NUM_EPOCHS,
    'BATCH_SIZE': BATCH_SIZE,
    'LEARNING_RATE': LEARNING_RATE,
}

In [ ]:
def resize(arr, new_size):
  return cv2.resize(arr, dsize=new_size, interpolation=cv2.INTER_NEAREST)


def normalize(dataset, max_val):
  dataset[:, 0] /= max_val
  dataset[:, 1][dataset[:, 1] > 0] = 1


def get_mask(filename):
  mask = []
  with open(filename, 'rb') as f:  # Читаем файл в бинарном режиме
    for l in f:  # Читаем построчно (по факту всего одна строка)
      sqrt_l = int(sqrt(len(l)))  # Получаем высоту и ширину маски (например, 1 строка 65536 символов - это квадратная маска 256х256=65536)
      # При этом полученные размеры маски вдвое больше размеров изображения (например, маска 256х256 для изображения 128х128)
      # Берем каждые sqrt_l*2 значений маски из строки и записываем в двумерный массив маски
      for i in range(int(sqrt_l/4)):
        for j in range(2):
          mask.append([int(s) for s in l[i*sqrt_l*4+j*sqrt_l*2:i*sqrt_l*4+j*sqrt_l*2+sqrt_l*2]])  # Одновременно переводим байти в int
      # В результате получаем маску: sqrt_l/2(высота) х sqrt_l*2(ширина)
      # Нужная высота уже есть (=sqrt_l/2), осталось получить нужную ширину (=sqrt_l/2)
  mask = np.array(mask)
  mask = np.delete(mask, range(0, sqrt_l*2, 2), 1)  # Удаляем каждый второй столбец маски с лишними нулями
  mask = np.delete(mask, range(0, sqrt_l, 2), 1)  # Удаляем каждый второй столбец маски с лишними нулями и значениями
  # *Это я дополнительно приводил все маски к одному размеру для обучения нейронки
  if mask.shape != (128, 128):
    mask = resize(mask, (128, 128))
  return mask


def getDataset(dirname):

  def get_set_name(dataset, curr_ds):
    new_len = sum(len(item) for item in dataset['data'].values()) + len(curr_ds)
    ideal = {'train': dataset['size']['train']*new_len,
             'validation': dataset['size']['validation']*new_len,
             'test': dataset['size']['test']*new_len}
    errors = {'train': (abs(len(dataset['data']['train']) + len(curr_ds) - ideal['train']) +
                        abs(len(dataset['data']['validation']) - ideal['validation']) +
                        abs(len(dataset['data']['test']) - ideal['test'])),
              'validation': (abs(len(dataset['data']['train']) - ideal['train']) +
                        abs(len(dataset['data']['validation']) + len(curr_ds) - ideal['validation']) +
                        abs(len(dataset['data']['test']) - ideal['test'])),
              'test': (abs(len(dataset['data']['train']) - ideal['train']) +
                        abs(len(dataset['data']['validation']) - ideal['validation']) +
                        abs(len(dataset['data']['test']) + len(curr_ds) - ideal['test']))}
    return min(errors, key=errors.get)

  dataset = {'data': {'train': [], 'validation': [], 'test': []},
            #  'size': {'train': 0.64, 'validation': 0.16, 'test': 0.2},
            #  'size': {'train': 0.72, 'validation': 0.13, 'test': 0.15},
             'size': {'train': 0.68, 'validation': 0.12, 'test': 0.2},
             'len': {'train': 0, 'validation': 0, 'test': 0},
             'file': {'train': [], 'validation': [], 'test': []}}
  for f in walk(dirname):  # Рекурсивно проходимся по всем папкам датасета
    if len(f[2]) > 0:  # Если папка содержит файлы
      dirpath = f[0]  # Записываем в переменную путь к файлам
      dirname = dirpath.split('/')[1]
      filenames = sorted(f[2])  # Сортируем имена файлов

      masknames = [fn for fn in filenames if '.txt' in fn]  # Записываем в переменную отсортированные имена масок в папке
      masks = {}
      if len(masknames) > 0:  # Если папка содержит маски
        for mn in masknames:  # Перебираем циклом имена масок
          idx = int(mn[:-4].split('-')[1])  # Получаем индекс маски из ее названия
          masks[idx] = get_mask(f'{dirpath}/{mn}')  # Добавляем маску в словарь
      else:
        continue

      dcm_groups = {}
      dcm_filenames = {}
      for fn in filenames:  # Перебираем циклом имена файлов
        if '.dcm' in fn:  # Отбираем только dcm-файлы
          dcm_file = pydicom.dcmread(f'{dirpath}/{fn}')  # Считываем и записываем dcm-файл в переменную
          if dcm_file.SliceLocation in dcm_groups.keys():  # Если словарь dcm_groups содержит ключь = значению тега SliceLocation данного dcm-файла
            dcm_groups[dcm_file.SliceLocation][dcm_file.InstanceNumber] = dcm_file
            dcm_filenames[dcm_file.SliceLocation][dcm_file.InstanceNumber] = fn
          else:
            dcm_groups[dcm_file.SliceLocation] = {dcm_file.InstanceNumber: dcm_file}  # Добавляем в словарь по ключу SliceLocation массив с dcm-файлом
            dcm_filenames[dcm_file.SliceLocation] = {dcm_file.InstanceNumber: fn}

      sorted_dcm_groups = sorted(dcm_groups)  # Сортируем ключи (SliceLocation) словаря dcm_groups
      sorted_dcm_filenames = sorted(dcm_filenames)

      curr_ds = []
      curr_len = 0
      cur_filenames = []
      for k in sorted_dcm_groups:  # Перебираем отсортированные ключи (SliceLocation)
        if sorted_dcm_groups.index(k) in masks.keys():  # Если к группе файлов есть маска
          curr_mask = masks[sorted_dcm_groups.index(k)]  # Записывам в переменную текущую маску
          flatten_mask = curr_mask.flatten()
          if flatten_mask.shape[0]*0.1 < np.count_nonzero(flatten_mask != flatten_mask[0]) < flatten_mask.shape[0]*0.9:  # Если нулевой элемент массива маски повторяется больше 10%, но меньше 90% (а то бывает, что выделено 2 пикселя и это типо маска - что явно не верно)
            k2 = sorted(dcm_groups[k])[3]
            curr_img = dcm_groups[k][k2].pixel_array  # Берем массив пикселей dcm-файла
            if curr_img.shape != (128, 128):  # Если размер не 128х128
              curr_img = resize(curr_img, (128, 128))  # Приводим к размеру 128х128
            curr_ds.insert(0, [curr_img, curr_mask])
            curr_len += 1
            cur_filenames.insert(0, f'{dirname}/{dcm_filenames[k][k2]}')

      set_name = get_set_name(dataset, curr_ds)
      dataset['data'][set_name] = curr_ds[:curr_len] + dataset['data'][set_name] + curr_ds[curr_len:]
      dataset['len'][set_name] += curr_len
      dataset['file'][set_name] = cur_filenames[:curr_len] + dataset['file'][set_name] + cur_filenames[curr_len:]

  for key in dataset['data'].keys():
    dataset['data'][key] = np.array(dataset['data'][key], dtype=float)

  return dataset


dataset = getDataset('TCGA-GBM-raw')
max_val = np.max(np.concatenate((dataset['data']['train'][:, 0],
                                 dataset['data']['validation'][:, 0],
                                 dataset['data']['test'][:, 0]), axis=0))
for key in dataset['data'].keys():
  normalize(dataset['data'][key], max_val)
  dataset['data'][key] = dataset['data'][key].reshape((*dataset['data'][key].shape, 1))
  print(dataset['data'][key].shape, dataset['len'][key], key, sep='\t')

In [ ]:
class CustomDataGen(Sequence):

    def __init__(self, dataset,
                 batch_size,
                 input_size=(128, 128, 1),
                 do_augment=False
                 ):
      self.ds = deepcopy(dataset)
      self.batch_size = batch_size
      self.input_size = input_size
      self.n = self.ds.shape[0]

      self.do_augment = do_augment;

    def on_epoch_end(self):
      np.random.shuffle(self.ds)

    def augment(self, batches):
      for i in range(batches.shape[0]):
        if random() > 0.5:
          batches[i, 0] = np.fliplr(batches[i, 0])
          batches[i, 1] = np.fliplr(batches[i, 1])
        if random() > 0.5:
          max_0 = np.max(batches[i, 0])
          angle = randint(-15, 15)
          batches[i, 0] = rotate(batches[i, 0], angle, reshape=False)
          batches[i, 1] = rotate(batches[i, 1], angle, reshape=False)
          # normalization
          batches[i, 0] = ((batches[i, 0] - np.min(batches[i, 0])) / (np.max(batches[i, 0]) - np.min(batches[i, 0])))*max_0
          batches[i, 1] = np.around((batches[i, 1] - np.min(batches[i, 1])) / (np.max(batches[i, 1]) - np.min(batches[i, 1])))

    def __getitem__(self, index):
      batches = deepcopy(self.ds[index*self.batch_size:(index + 1)*self.batch_size])
      if self.do_augment:
        self.augment(batches)  # аугментация
      X = batches[:, 0]
      y = batches[:, 1]
      return X, y

    def __len__(self):
      return int(self.n // self.batch_size)


train_gen = CustomDataGen(dataset['data']['train'], BATCH_SIZE, do_augment=True)
validation_gen = CustomDataGen(dataset['data']['validation'], BATCH_SIZE)
test_gen = CustomDataGen(dataset['data']['test'], dataset['len']['test'])

In [ ]:
def sSE_block(x):

  x_1 = x
  x = layers.Conv2D(1, 1, activation="sigmoid")(x)
  x = layers.multiply([x, x_1])

  return x


def cSE_block(x, n_filters):

  x_1 = x
  x = layers.GlobalAveragePooling2D(keepdims=True)(x)
  x = layers.Conv2D(n_filters/2, 1, activation="relu")(x)
  x = layers.Conv2D(n_filters, 1, activation="sigmoid")(x)
  x = layers.multiply([x, x_1])

  return x


def scSE_block(x, n_filters):

  sSE = sSE_block(x)
  cSE = cSE_block(x, n_filters)
  x = layers.add([sSE, cSE])

  return x


def downsample_block(x, n_filters):

    x_1 = layers.Conv2D(1, 1, activation="relu")(x)
    x_1 = layers.MaxPool2D(2)(x_1)
    x = layers.Conv2D(n_filters, 3, 2, padding="same", activation="relu")(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = layers.add([x, x_1])

    x_2 = scSE_block(x, n_filters) if CASE_SE == 'scSE-Identity' else x  # scSE-Identity
    x = scSE_block(x, n_filters) if CASE_SE == 'scSE-PRE' else x  # scSE-PRE
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = scSE_block(x, n_filters) if CASE_SE == 'Standard-scSE' else x  # Standard-scSE
    x = layers.add([x, x_2])
    x = scSE_block(x, n_filters) if CASE_SE == 'scSE-POST' else x  # scSE-POST

    f = x
    p = x

    return f, p


def upsample_block(x, conv_features, n_filters):

    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    x = layers.concatenate([x, conv_features])

    x_1 = layers.Conv2D(1, 1, activation="relu")(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = layers.add([x, x_1])

    x_2 = scSE_block(x, n_filters) if CASE_SE == 'scSE-Identity' else x  # scSE-Identity
    x = scSE_block(x, n_filters) if CASE_SE == 'scSE-PRE' else x  # scSE-PRE
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    x = scSE_block(x, n_filters) if CASE_SE == 'Standard-scSE' else x  # Standard-scSE
    x = layers.add([x, x_2])
    x = scSE_block(x, n_filters) if CASE_SE == 'scSE-POST' else x  # scSE-POST

    return x


def build_model():

    # inputs
    inputs = layers.Input(shape=(128, 128, 1))

    f0 = layers.Conv2D(64, 7, 2, padding='same', activation="relu")(inputs)
    p0 = f0

    # encoder: contracting path - downsample
    f1, p1 = downsample_block(p0, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)

    # decoder: expanding path - upsample
    u5 = upsample_block(p4, f3, 512)
    u6 = upsample_block(u5, f2, 256)
    u7 = upsample_block(u6, f1, 128)

    u8 = layers.Conv2DTranspose(64, 3, 2, padding="same")(u7)
    u8 = layers.concatenate([u8, f0])
    u8 = layers.Conv2DTranspose(64, 7, 2, padding="same")(u8)

    # outputs
    outputs = layers.Conv2D(2, 1, padding="same", activation="softmax")(u8)

    model = Model(inputs, outputs)

    return model

In [ ]:
scc = SparseCategoricalCrossentropy()
val_loss_begin = []
val_loss_begin_clbk = LambdaCallback(on_epoch_begin=lambda epoch, logs:
                                     val_loss_begin.append(scc(dataset['data']['validation'][:, 1],
                                                               model.predict(dataset['data']['validation'][:, 0])).numpy()))

model = build_model()

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss=scc,
              metrics=['sparse_categorical_accuracy'],
              )

model_history = model.fit(
    x=train_gen,
    validation_data=validation_gen,
    epochs=NUM_EPOCHS,
    callbacks=[val_loss_begin_clbk,
               ReduceLROnPlateau(factor=0.1, patience=10),
               ModelCheckpoint(
                   filepath=f'{RES_FOLDER}/best_model.keras',
                   save_best_only=True)
               ])

model = tf.keras.models.load_model(f'{RES_FOLDER}/best_model.keras')

In [ ]:
model.summary()

In [ ]:
def get_stats(true_mask, pred_mask):
  tp, tn, fp, fn = 0, 0, 0, 0
  for i in range(pred_mask.shape[0]):
    for j in range(pred_mask.shape[1]):
      if pred_mask[i, j] == true_mask[i, j]:
        if pred_mask[i, j] == 0:
          tn += 1
        else:
          tp += 1
      else:
        if pred_mask[i, j] == 0:
          fn += 1
        else:
          fp += 1
  return tp, tn, fp, fn


def get_dice(tp, tn, fp, fn):
  return 2*tp / (2*tp + fp + fn)


def get_sens(tp, tn, fp, fn):
  return tp / (tp + fn)


def get_spec(tp, tn, fp, fn):
  return tn / (tn + fp)


def get_accu(tp, tn, fp, fn):
  return (tp + tn) / (tp + tn + fp + fn)


def get_metrics(tp, tn, fp, fn):
  dice = get_dice(tp, tn, fp, fn)
  sens = get_sens(tp, tn, fp, fn)
  spec = get_spec(tp, tn, fp, fn)
  accu = get_accu(tp, tn, fp, fn)
  return dice, sens, spec, accu


def create_mask(pred_mask):
  pred_mask = np.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., np.newaxis]
  return pred_mask[1]


def true_pred_mask(true_mask, pred_mask):
  true_pred_mask = deepcopy(pred_mask)
  for i in range(pred_mask.shape[0]):
    for j in range(pred_mask.shape[1]):
      if true_mask[i, j] == 0:
        true_pred_mask[i, j] = 2 if pred_mask[i, j] == 1 else 0
      else:  # 1
        true_pred_mask[i, j] = 3 if pred_mask[i, j] == 0 else 1
  palette = np.array([[  0,   0,   0],   # tn - black
                      [255,   0,   0],   # tp - red
                      [  0, 255,   0],   # fp - green
                      [  0,   0, 255]],  # fn - blue
                      dtype='uint8')
  return palette[true_pred_mask][..., np.newaxis]


def show_predictions(dataset):
  for d in dataset:
    pred_mask = create_mask(model.predict(d, verbose=0))
    fig = plt.figure(figsize=(10, 5))
    i = 0
    titles = ['input image', 'true mask', 'pred mask', 'tn(0) tp(r) fp(g) fn(b)']
    for img in [d[0], d[1], pred_mask, true_pred_mask(d[1], pred_mask)]:
      fig.add_subplot(1, 4, i+1)
      plt.imshow(np.squeeze(img))
      plt.axis('off')
      plt.title(titles[i])
      i += 1
    plt.show()
    tp, tn, fp, fn = get_stats(d[1], pred_mask)
    print(f'TP = {tp}, TN = {tn}, FP = {fp}, FN = {fn}')
    print(f'DICE = {get_dice(tp, tn, fp, fn)}')
    print(f'SENS = {get_sens(tp, tn, fp, fn)}')
    print(f'SPEC = {get_spec(tp, tn, fp, fn)}')
    print(f'ACCU = {get_accu(tp, tn, fp, fn)}\n')


show_predictions(dataset['data']['test'][:1])

In [ ]:
def save_results(dataset):

  def add_channels(img, imgtype='mask'):
    channels_img = np.zeros((img.shape[0], img.shape[1], 3), dtype='uint8')
    if imgtype == 'mask':
      channels_img[img[:, :, 0] == 1] = np.array([255, 0, 0], dtype='uint8')
    elif imgtype == 'img':
      img_max = np.amax(img)
      for x in range(img.shape[0]):
        for y in range(img.shape[1]):
          for z in range(channels_img.shape[2]):
            channels_img[x, y, z] = round(255*(img[x, y, 0] / img_max))
    return channels_img

  def save_accuracy():
    plt.rcParams.update({'font.size': 14})
    figure(figsize=(25, 10))
    val_accuracy = model_history.history['val_sparse_categorical_accuracy']
    max_val_accuracy = max(val_accuracy)
    best_accuracy = plt.scatter(val_accuracy.index(max_val_accuracy)+1, max_val_accuracy, s=100, c='red', zorder=3)
    t_accuracy = plt.plot(range(1, NUM_EPOCHS+1), model_history.history['sparse_categorical_accuracy'], 'b^-', zorder=1)
    v_accuracy = plt.plot(range(1, NUM_EPOCHS+1), val_accuracy, 'gs-', zorder=2)
    plt.xticks(range(0, NUM_EPOCHS+1, 5))
    plt.title(CASE_SE)
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend((t_accuracy[0], v_accuracy[0], best_accuracy),
               ('train accuracy', 'validation accuracy', 'best model'),
               loc='lower right')
    plt.savefig(f'{RES_FOLDER}/accuracy.png')

  def save_loss():
    plt.rcParams.update({'font.size': 14})
    figure(figsize=(25, 10))
    val_loss = model_history.history['val_loss']
    min_val_loss = min(val_loss)
    best_loss = plt.scatter(val_loss.index(min_val_loss)+1, min_val_loss, s=100, c='red', zorder=3)
    t_loss = plt.plot(range(1, NUM_EPOCHS+1), model_history.history['loss'], 'b^-', zorder=1)
    v_loss = plt.plot(range(1, NUM_EPOCHS+1), val_loss, 'gs-', zorder=2)
    plt.xticks(range(0, NUM_EPOCHS+1, 5))
    plt.title(CASE_SE)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend((t_loss[0], v_loss[0], best_loss),
               ('train loss', 'validation loss', 'best model'),
               loc='upper right')
    plt.savefig(f'{RES_FOLDER}/loss.png')

  # val_loss_begin
  def save_loss_begin():
    plt.rcParams.update({'font.size': 14})
    figure(figsize=(25, 10))
    val_loss = val_loss_begin
    min_val_loss = min(val_loss)
    best_loss = plt.scatter(val_loss.index(min_val_loss)+1, min_val_loss, s=100, c='red', zorder=3)
    t_loss = plt.plot(range(1, NUM_EPOCHS+1), model_history.history['loss'], 'b^-', zorder=1)
    v_loss = plt.plot(range(1, NUM_EPOCHS+1), val_loss, 'gs-', zorder=2)
    plt.xticks(range(0, NUM_EPOCHS+1, 5))
    plt.title(CASE_SE)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend((t_loss[0], v_loss[0], best_loss),
               ('train loss', 'validation loss', 'best model'),
               loc='upper right')
    plt.savefig(f'{RES_FOLDER}/loss_begin.png')

  save_accuracy()
  save_loss()
  save_loss_begin()  # val_loss_begin
  workbook = xlsxwriter.Workbook(f'{RES_FOLDER}/results.xlsx')
  workbook.add_worksheet(CASE_SE)
  worksheet = workbook.get_worksheet_by_name(CASE_SE)
  sum_TP, sum_TN, sum_FP, sum_FN = 0, 0, 0, 0
  row, col = 0, 0
  for k, v in HP.items():
    worksheet.write_row(row, col, [k, v])
    row += 1
  row += 1
  worksheet.write_row(row, col, ['epoch', *range(1, NUM_EPOCHS+1)])
  row += 1
  for k, v in model_history.history.items():
    worksheet.write_row(row, col, [k, *v])
    row += 1
  worksheet.write_row(row, col, ['val_loss_begin', *val_loss_begin])  # val_loss_begin
  row += 2
  worksheet.write_row(row, col, ['TP', 'TN', 'FP', 'FN', 'DICE', 'SENS', 'SPEC', 'ACCU', 'FILE'])
  row += 1
  img_idx = 0
  dice_vals, sens_vals, spec_vals, accu_vals = np.array([]), np.array([]), np.array([]), np.array([])
  for i in range(dataset['len']['test']):
    d = dataset['data']['test'][i]
    pred_mask = create_mask(model.predict(d, verbose=0))
    # Save imgs
    mpimg.imsave(f'{RES_FOLDER}/imgs/{img_idx}__input_image.png', add_channels(d[0], 'img'))
    mpimg.imsave(f'{RES_FOLDER}/imgs/{img_idx}__true_mask.png', add_channels(d[1]))
    mpimg.imsave(f'{RES_FOLDER}/imgs/{img_idx}__pred_mask.png', add_channels(pred_mask))
    mpimg.imsave(f'{RES_FOLDER}/imgs/{img_idx}__tn(0)_tp(r)_fp(g)_fn(b).png',
                 np.squeeze(true_pred_mask(d[1], pred_mask)))
    img_idx += 1
    tp, tn, fp, fn = get_stats(d[1], pred_mask)
    dice, sens, spec, accu = get_metrics(tp, tn, fp, fn)
    dice_vals = np.append(dice_vals, dice)
    sens_vals = np.append(sens_vals, sens)
    spec_vals = np.append(spec_vals, spec)
    accu_vals = np.append(accu_vals, accu)
    sum_TP += tp
    sum_TN += tn
    sum_FP += fp
    sum_FN += fn
    worksheet.write_row(row, col, [tp, tn, fp, fn, dice, sens, spec, accu, dataset['file']['test'][i]])
    row += 1
  row += 1
  worksheet.write_row(row, col, ['', 'DICE', 'SENS', 'SPEC', 'ACCU'])
  row += 1
  worksheet.write_row(row, col, ['mean', np.mean(dice_vals), np.mean(sens_vals), np.mean(spec_vals), np.mean(accu_vals)])
  row += 1
  worksheet.write_row(row, col, ['std', np.std(dice_vals), np.std(sens_vals), np.std(spec_vals), np.std(accu_vals)])
  row += 1
  worksheet.write_row(row, col, ['min', np.min(dice_vals), np.min(sens_vals), np.min(spec_vals), np.min(accu_vals)])
  row += 1
  worksheet.write_row(row, col, ['max', np.max(dice_vals), np.max(sens_vals), np.max(spec_vals), np.max(accu_vals)])
  row += 2
  worksheet.write(row, col, 'TOTAL:')
  row += 1
  worksheet.write_row(row, col, ['TP', 'TN', 'FP', 'FN', 'DICE', 'SENS', 'SPEC', 'ACCU'])
  row += 1
  sum_DICE, sum_SENS, sum_SPEC, sum_ACCU = get_metrics(sum_TP, sum_TN, sum_FP, sum_FN)
  worksheet.write_row(row, col, [sum_TP, sum_TN, sum_FP, sum_FN, sum_DICE, sum_SENS, sum_SPEC, sum_ACCU])
  workbook.close()


save_results(dataset)